# Data Management
Use this file to import and manipulate the data before saving them to the database.

In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import requests

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import desc

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean

# Local modules
from config import omdb_api_key

## Movies, actor and character database
The CSV files were downloaded from https://data.world/jamesgaskin/movies (author: James Gaskin). The files are opened here to prepare the database.

In [2]:
# Get the movies data
movies_csv = Path('../Datasets/movies.csv')
movies_df = pd.read_csv(movies_csv)

In [3]:
# List the columns in the DataFrame
movies_df.columns

Index(['movieid', 'title', 'mpaa_rating', 'budget', 'gross', 'release_date',
       'genre', 'runtime', 'rating', 'rating_count', 'summary'],
      dtype='object')

In [4]:
# Get all movies in the dataset into a list
movie_list = movies_df['title'].tolist()
print(f"There are {len(movie_list)} movies in the list.")

There are 636 movies in the list.


In [5]:
characters_df = Path('../Datasets/character.csv')
characters_df = pd.read_csv(characters_df)

In [6]:
characters_df

,movieid,actorid,character_name,creditorder,pay,screentime
0,1,1,James,1,NaN,NaN
1,1,2,Mollie,2,NaN,NaN
2,1,3,Rosie,3,NaN,NaN
3,1,4,Albert,4,NaN,NaN
4,1,5,Grandpa,5,NaN,NaN
...,...,...,...,...,...,...
4314,483,2494,Explorer Captain,6,NaN,NaN
4315,483,2495,Russian Space Station Captain (voice),7,NaN,NaN
4316,483,2495,Russian Space Station Captain (voice),8,NaN,NaN
4317,483,2495,Russian Space Station Captain (voice),9,NaN,NaN


In [7]:
actor_df = Path('../Datasets/actor.csv')
actor_df = pd.read_csv(actor_df)

In [8]:
actor_df

,actorid,name,date_of_birth,birth_city,birth_country,height_inches,biography,gender,ethnicity,networth
0,2169,Aaron Eckhart,1968-03-12,Cupertino,USA,61.0,"Aaron Eckhart was born on March 12, 1968 in Cu...",Male,White,16000000.0
1,2204,Aasif Mandvi,1966-03-05,Bombay,India,68.0,"Aasif Mandvi was born on March 5, 1966 in Bomb...",Male,Indian,3000000.0
2,5,Abe Vigoda,1921-02-24,Brooklyn,USA,73.0,Abraham Charles Vigoda was an American actor k...,Male,White,10000000.0
3,1618,Abigail Breslin,1996-04-14,New York City,USA,61.0,Academy Award-nominated actress Abigail Bresli...,Female,White,8000000.0
4,1062,Abraham Benrubi,1969-10-04,Indianapolis,USA,79.0,"Abraham Benrubi was born on October 4, 1969 in...",Male,White,12000000.0
...,...,...,...,...,...,...,...,...,...,...
2594,1337,Ziyi Zhang,1979-02-09,Beijing,China,65.0,Ziyi Zhang is a Chinese actress and model. She...,NaN,NaN,NaN
2595,2566,Zoe Kravitz,1988-12-01,Los Angeles,USA,62.0,"Zoe Isabella Kravitz, the daughter of singer/a...",Female,NaN,NaN
2596,2232,Zoe Saldana,1978-06-19,Passaic,USA,66.0,"Zoe Saldana was born on June 19, 1978 in Passa...",Female,NaN,35000000.0
2597,2534,Zoey Vargas,NaN,NaN,NaN,NaN,"Zoey Vargas is an actress, known for Neighbors...",Female,NaN,NaN


## OMDB
NOTE: Before proceeding please make sure that the repo contains `config.py` with a variable `omdb_api_key` that contains your OMDB API.

This section of the code loop through the movies included in the James Gaskin dataset and retrieve all their information from OMDB.

In [12]:
# Base URL and key
url = "http://www.omdbapi.com/?"
api_key = "&apikey=" + omdb_api_key

# Initialise list and movie counter
omdb_movies = []
counter = 0

# Loop through the list of movie titles
for movie in movie_list:

    # Display progress counter
    counter += 1
    print(f"Get info for Movie {counter} of {len(movie_list)}: {movie}")

    try:
        # Add movie title to search item and contruct API URL
        movie_title = f"t={movie}"
        url_omdb = url + movie_title + api_key

        # Add movie to list
        omdb_movies.append(requests.get(url_omdb).json())
    except:
        print("Movie not found.")


Get info for Movie 1 of 636: Look Who's Talking
Get info for Movie 2 of 636: Driving Miss Daisy
Get info for Movie 3 of 636: Turner & Hooch
Get info for Movie 4 of 636: Born on the Fourth of July
Get info for Movie 5 of 636: Field of Dreams
Get info for Movie 6 of 636: Uncle Buck
Get info for Movie 7 of 636: When Harry Met Sally...
Get info for Movie 8 of 636: Dead Poets Society
Get info for Movie 9 of 636: Parenthood
Get info for Movie 10 of 636: Lethal Weapon 2
Get info for Movie 11 of 636: The War of the Roses
Get info for Movie 12 of 636: National Lampoon's Christmas Vacation
Get info for Movie 13 of 636: Honey, I Shrunk the Kids
Get info for Movie 14 of 636: Batman
Get info for Movie 15 of 636: Ghostbusters II
Get info for Movie 16 of 636: The Little Mermaid
Get info for Movie 17 of 636: Back to the Future Part II
Get info for Movie 18 of 636: Indiana Jones and the Last Crusade
Get info for Movie 19 of 636: Tango & Cash
Get info for Movie 20 of 636: Steel Magnolias
Get info for Mo

In [15]:
# Save the results in a DataFrame
omdb_movies_df = pd.DataFrame(omdb_movies)

# Save the DataFrame to a CSV file
# omdb_movies_df.to_csv('Datasets/omdb.csv', index=False)

In [9]:
# List the columns in the DataFrame
movies_df.columns

Index(['movieid', 'title', 'mpaa_rating', 'budget', 'gross', 'release_date',
       'genre', 'runtime', 'rating', 'rating_count', 'summary'],
      dtype='object')

In [17]:
# List the columns in the DataFrame
omdb_movies_df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response', 'Error',
       'totalSeasons'],
      dtype='object')

## Adding data to SQLite db

## Add movies.csv to database

In [2]:
# Get the movies data
movies_csv = Path('../Datasets/movies.csv')
movies_df = pd.read_csv(movies_csv)

In [3]:
movies_df.dtypes

movieid           int64
title            object
mpaa_rating      object
budget           object
gross            object
release_date     object
genre            object
runtime          object
rating          float64
rating_count    float64
summary          object
dtype: object

In [4]:
Base = declarative_base()

# Create movies class
class Movies(Base):
    __tablename__ = 'movies'
    movieid = Column(Integer, primary_key=True)
    title = Column(String)
    mpaa_rating = Column(String)
    budget = Column(String)
    gross = Column(String)
    release_date = Column(String)
    genre = Column(String)
    runtime = Column(Integer)
    rating = Column(Float)
    rating_count = Column(Float)
    summary = Column(String)                       

In [5]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///movies_db.sqlite')

# Create the tables within the database
Base.metadata.create_all(engine)
# Start session
session = Session(bind=engine)

In [6]:
# Loop through DataFrame and retrieve data
for index, row in movies_df.iterrows():
    movieid = int(movies_df.loc[index,'movieid'])
    title = movies_df.loc[index,'title']
    mpaa_rating = movies_df.loc[index,'mpaa_rating']
    budget = movies_df.loc[index,'budget']
    gross = movies_df.loc[index,'gross']
    release_date = movies_df.loc[index,'release_date']
    genre = movies_df.loc[index,'genre']
    runtime = movies_df.loc[index,'runtime']
    rating = movies_df.loc[index,'rating']
    rating_count = movies_df.loc[index,'rating_count']
    # summary = movies_df.loc[index,'summary']
    print(title)

    # Add data to database
    session.add(Movies(
        movieid = movieid,
        title = title,
        mpaa_rating = mpaa_rating,
        budget = budget,
        gross = gross,
        release_date = release_date,
        genre = genre,
        runtime = runtime,
        rating = rating,
        rating_count = rating_count
        # summary = summary
    ))

    session.commit()
    
print("ok")


Look Who's Talking
Driving Miss Daisy
Turner & Hooch
Born on the Fourth of July
Field of Dreams
Uncle Buck
When Harry Met Sally...
Dead Poets Society
Parenthood
Lethal Weapon 2
The War of the Roses
National Lampoon's Christmas Vacation
Honey, I Shrunk the Kids
Batman
Ghostbusters II
The Little Mermaid
Back to the Future Part II
Indiana Jones and the Last Crusade
Tango & Cash
Steel Magnolias
Three Men and a Little Lady
Teenage Mutant Ninja Turtles
Pretty Woman
Kindergarten Cop
Home Alone
Bird on a Wire
Misery
Edward Scissorhands
Ghost
Presumed Innocent
Dances with Wolves
Flatliners
The Hunt for Red October
Another 48 Hrs.
Back to the Future Part III
Dick Tracy
The Godfather: Part III
Days of Thunder
Total Recall
Die Hard 2
Fried Green Tomatoes
The Silence of the Lambs
Sleeping with the Enemy
Father of the Bride
The Naked Gun 2 1/2: The Smell of Fear
Teenage Mutant Ninja Turtles II
Beauty and the Beast
Hot Shots!
City Slickers
Star Trek VI: The Undiscovered Country
The Addams Family
The 

In [8]:
# Commit all changes to database
session.commit()

IntegrityError: (sqlite3.IntegrityError) datatype mismatch
[SQL: INSERT INTO movies (movieid, title, mpaa_rating, budget, gross, release_date, genre, runtime, rating, rating_count, summary) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((1, "Look Who's Talking", 'PG-13', '7500000', '296000000', '1989-10-12', 'Romance', '93', 5.9, 73638.0, 'After a single, career-minded woman is left on her own to give birth to the child of a married man, she finds a new romantic chance in a cab driver. Meanwhile, the point-of-view of the newborn boy is narrated through voice-over.'), (2, 'Driving Miss Daisy', 'PG', '7500000', '145793296', '1989-12-13', 'Comedy', '99', 7.4, 91075.0, 'An old Jewish woman and her African-American chauffeur in the American South have a relationship that grows and improves over the years.'), (3, 'Turner & Hooch', 'PG', '13000000', '71079915', '1989-07-28', 'Crime', '100', 7.2, 91415.0, "Det. Scott Turner (Tom Hanks) is an uptight, by-the-book police officer. When h

## Get the actor into the database

In [20]:
# Create actor class
class Actors(Base):
    __tablename__ = 'actor'
    actorid = Column(Integer, primary_key=True)
    name = Column(String)
    date_of_birth = Column(String)
    birth_city = Column(String)
    birth_country = Column(String)
    height_inches = Column(Integer)
    biography = Column(String) 
    gender = Column(String)
    ethnicity = Column(String)
    networth = Column(Integer)

In [21]:
# Create omdb class
class Omdb(Base):
    __tablename__ = 'omdb'
    id = Column(Integer, primary_key=True)
    Title = Column(String)
    Year = Column(Integer)
    Rated = Column(String)
    Released = Column(String)
    Runtime = Column(String)
    Genre = Column(String)
    Director = Column(String)   
    Writer = Column(String)
    Actors = Column(String)
    Plot = Column(String)
    Language = Column(String)
    Country = Column(String)
    Awards = Column(String)
    Poster = Column(String)
    Ratings = Column(String)
    Metascore = Column(Integer)
    imdbRating = Column(Float)
    imdbVotes = Column(Integer)
    imdbID = Column(String)
    Type = Column(String)
    DVD = Column(String)
    BoxOffice = Column(Integer)
    Production = Column(String)
    Website = Column(String)
    Response = Column(Boolean)
    Error = Column(String)
    totalSeasons = Column(Integer)

In [22]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///movies_db.sqlite')

In [23]:
# Create the tables within the database
Base.metadata.create_all(engine)
# Start session
session = Session(bind=engine)

In [24]:
# Loop through DataFrame and retrieve data
for index, row in movies_df.iterrows():
    movieid = movies_df.loc[index,'movieid']
    title = movies_df.loc[index,'title']
    mpaa_rating = movies_df.loc[index,'mpaa_rating']
    budget = movies_df.loc[index,'budget']
    gross = movies_df.loc[index,'gross']
    release_date = movies_df.loc[index,'release_date']
    genre = movies_df.loc[index,'genre']
    runtime = movies_df.loc[index,'runtime']
    rating = movies_df.loc[index,'rating']
    rating_count = movies_df.loc[index,'rating_count']
    summary = movies_df.loc[index,'summary']
    print(title)

    # Add data to database
    session.add(Movies(
        movieid = movieid,
        title = title,
        mpaa_rating = mpaa_rating,
        budget = budget,
        gross = gross,
        release_date = release_date,
        genre = genre,
        runtime = runtime,
        rating = rating,
        rating_count = rating_count,
        summary = summary

    ))
print("ok")


Look Who's Talking
Driving Miss Daisy
Turner & Hooch
Born on the Fourth of July
Field of Dreams
Uncle Buck
When Harry Met Sally...
Dead Poets Society
Parenthood
Lethal Weapon 2
The War of the Roses
National Lampoon's Christmas Vacation
Honey, I Shrunk the Kids
Batman
Ghostbusters II
The Little Mermaid
Back to the Future Part II
Indiana Jones and the Last Crusade
Tango & Cash
Steel Magnolias
Three Men and a Little Lady
Teenage Mutant Ninja Turtles
Pretty Woman
Kindergarten Cop
Home Alone
Bird on a Wire
Misery
Edward Scissorhands
Ghost
Presumed Innocent
Dances with Wolves
Flatliners
The Hunt for Red October
Another 48 Hrs.
Back to the Future Part III
Dick Tracy
The Godfather: Part III
Days of Thunder
Total Recall
Die Hard 2
Fried Green Tomatoes
The Silence of the Lambs
Sleeping with the Enemy
Father of the Bride
The Naked Gun 2 1/2: The Smell of Fear
Teenage Mutant Ninja Turtles II
Beauty and the Beast
Hot Shots!
City Slickers
Star Trek VI: The Undiscovered Country
The Addams Family
The 

In [25]:
# Commit all changes to database
session.commit()

IntegrityError: (sqlite3.IntegrityError) datatype mismatch
[SQL: INSERT INTO movies (movieid, title, mpaa_rating, budget, gross, release_date, genre, runtime, rating, rating_count, summary) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((1, "Look Who's Talking", 'PG-13', '7500000', '296000000', '1989-10-12', 'Romance', '93', 5.9, 73638.0, 'After a single, career-minded woman is left on her own to give birth to the child of a married man, she finds a new romantic chance in a cab driver. Meanwhile, the point-of-view of the newborn boy is narrated through voice-over.'), (2, 'Driving Miss Daisy', 'PG', '7500000', '145793296', '1989-12-13', 'Comedy', '99', 7.4, 91075.0, 'An old Jewish woman and her African-American chauffeur in the American South have a relationship that grows and improves over the years.'), (3, 'Turner & Hooch', 'PG', '13000000', '71079915', '1989-07-28', 'Crime', '100', 7.2, 91415.0, "Det. Scott Turner (Tom Hanks) is an uptight, by-the-book police officer. When h

In [26]:
# Loop through DataFrame and retrieve data
for index, row in actor_df.iterrows():
    actorid = actor_df.loc[index,'actorid']
    name = actor_df.loc[index,'name']
    date_of_birth = actor_df.loc[index,'date_of_birth']
    birth_city = actor_df.loc[index,'birth_city']
    birth_country = actor_df.loc[index,'birth_country']
    height_inches = actor_df.loc[index,'height_inches']
    biography = actor_df.loc[index,'biography']
    gender = actor_df.loc[index,'gender']
    ethnicity = actor_df.loc[index,'ethnicity']
    networth = actor_df.loc[index,'networth']
    print(name)

    # Add data to database
    session.add(Actors(
        actorid = actorid,
        name = name,
        date_of_birth = date_of_birth,
        birth_city = birth_city,
        birth_country = birth_country,
        height_inches = height_inches,
        biography = biography,
        gender = gender,
        ethnicity = ethnicity,
        networth = networth

    ))
print("ok")

Aaron Eckhart
Aasif Mandvi
Abe Vigoda
Abigail Breslin
Abraham Benrubi
Adam Arkin
Adam Ferrara
Adam Goldberg
Adam Hann-Byrd
Adam Sandler
Adelaide Clemens
Adewale Akinnuoye-Agbaje
Adoni Maropis
Adrian Dunbar
Adrian Grenier
Adrian Rawlins
Adrian Ricard
Adrian Scarborough
Adriane Lenox
Adrien Brody
Aedin Mincks
Aeryk Egan
Ahmed Best
Ahna O'Reilly
Ahney Her
Aidan Quinn
Al Gore
Al Pacino
Alain Chabat
Aaron Michael Drozin
Aaron Taylor-Johnson
Allen Payne
Allison Janney
Allison Karman
Alun Armstrong
Alvin M. Sugarman
Alyson Hannigan
Alyson Stoner
Amanda Bynes
Amanda Donohoe
Amanda Farinos
Amanda Peet
Amanda Plummer
Amanda Seyfried
Amaury Nolasco
Amber Scott
Amber Stevens West
Amber Tamblyn
Amber Valletta
America Ferrera
Amitabh Bachchan
Amy Adams
Amy Hill
Amy Madigan
Amy O'Neill
Amy Poehler
Amy Sedaris
Amy Warren
Amy Yasbeck
Andie MacDowell
Andre B. Blake
Andre Bernard
Andre Braugher
Andrea Martin
Andreas Katsulas
Andrew Bryniarski
Andrew Chavez
Andrew Divoff
Andrew Friedman
Andrew Garfield
An

Elise Vargas
Eliza Dushku
Elizabeth Banks
Elizabeth Daily
Elizabeth Lawrence
Elizabeth Mitchell
Elizabeth Olsen
Elizabeth Perkins
Elizabeth Reaser
Elizabeth Taylor
Elizabeth Wilson
Ella Mitchell
Ellen Albertini Dow
Ellen Burstyn
Ellen DeGeneres
Ellen Page
Elsie Fisher
Elyes Gabel
Embeth Davidtz
Emiliano Guerra
Emilio Rivera
Emily Ann Lloyd
Emily Blunt
Emily Browning
Emily Mortimer
Emma Stone
Emma Thompson
Emma Watson
Emmanuelle Bart
Emmy Rossum
Eric Bana
Eric Gordon
Eric Idle
Eric Lloyd
Eric Roberts
Eric Schweig
Eric Sykes
Erick Avari
Erik von Detten
Erika Christensen
Ernest Harden Jr.
Ernie Hudson
Estella Warren
Estelle Harris
Ester Dean
Esther Rolle
Ethan Embry
Ethan Hawke
Ethan Suplee
Etienne Chicot
Eugene Levy
Eunice Cho
Eva Green
Eva Marie Saint
Eva Mendes
Evan Handler
Evan Parke
Evanna Lynch
Ewan McGregor
Ewen Bremner
Fa Zeng Li
Fairuza Balk
Faith Ford
Faith Prince
Faizon Love
Famke Janssen
Fana Mokoena
Faran Tahir
Faune Chambers Watkins
Fiona Hogan
Fiona Shaw
Floyd 'Red Crow' We

Karen Finley
Karen Gillan
Karen Kruper
Karin Konoval
Karina Lombard
Karl Urban
Karolina Kurkova
Kasi Lemmons
Kat Dennings
Kate Beahan
Kate Beckinsale
Kate Bosworth
Kate McGregor-Stewart
Kate Nelligan
Kate Winslet
Kath Soucie
Katharine Houghton
Katharine Towne
Katherine Heigl
Kathleen Quinlan
Kathleen Turner
Kathryn Erbe
Kathryn Hunter
Kathy Baker
Kathy Bates
Kathy Najimy
Katie Holmes
Katie Sagona
Kaylee Vieira
Keanu Reeves
Keir O'Donnell
Keira Knightley
Keith Campbell
Keith David
Keith Ferguson
Keith Lucas
Keith Richards
Keith Robinson
Kellan Lutz
Kelley
Kelly Keaton
Kelly Lynch
Kelly Macdonald
Kelly Miracco
Kelly Preston
Kelly Reilly
Kelsey Grammer
Ken Davitian
Ken Jeong
Ken Leung
Ken Stott
Ken Watanabe
Kenneth Branagh
Kenneth Lucas
Kenneth Mars
Kenneth Tsang
Keri Russell
Kerry Washington
Kevin Anderson
Kevin Bacon
Kevin Costner
Kevin Dunn
Kevin Durand
Kevin G. Schmidt
Kevin J. O'Connor
Kevin James
Kevin Kline
Kevin McBride
Kevin McKidd
Kevin McNally
Kevin Patrick Walls
Kevin Pollak
K

Peter Riegert
Peter Sarsgaard
Peter Scolari
Peter Siragusa
Peter Sohn
Peter Stormare
Peter Von Berg
Peter Weireter
Petr Jakl
Petra Wildgoose
Phaldut Sharma
Phil Daniels
Phil LaMarr
Phil Proctor
Philip Baker Hall
Philip Bosco
Philip Seymour Hoffman
Phyllis Applegate
Phyllis Diller
Pierce Brosnan
Piper Perabo
Polly Holliday
Polly Walker
Powers Boothe
Queen Latifah
Quinton Aaron
R.D. Call
Rachael Bella
Rachael Harris
Rachael Taylor
Rachel Blackman
Rachel Covey
Rachel Griffiths
Rachel Majorowski
Rachel McAdams
Rachel McDowall
Rachel Ticotin
Rachel Weisz
Rade Serbedzija
Raini Rodriguez
Rainn Wilson
Raj Zutshi
Ralph Bellamy
Ralph Fiennes
Ralph Riach
Ralph Waite
Ramon Rodriguez
Randall Duk Kim
Randy Means
Randy Quaid
Randy Thom
Rasila Schroeder
Raul Julia
Raven-Symon
Rawle D. Lewis
Ray Anthony
Ray Godshall Sr.
Ray LePere
Ray Liotta
Ray McKinnon
Ray Park
Ray Romano
Ray Stevenson
Ray Winstone
Ray Wise
Raymond J. Barry
Raynor Scheine
Rebecca De Mornay
Rebecca Hall
Rebecca McFarland
Rebecca Romij

In [32]:
# Loop through DataFrame and retrieve data
for index, row in omdb_movies_df.iterrows():   
    id = index
    Title = omdb_movies_df.loc[index,'Title']
    Year = omdb_movies_df.loc[index,'Year']
    Rated = omdb_movies_df.loc[index,'Rated']
    Released = omdb_movies_df.loc[index,'Released']
    Runtime = omdb_movies_df.loc[index,'Runtime']
    Genre = omdb_movies_df.loc[index,'Genre']
    Director = omdb_movies_df.loc[index,'Director']
    Writer = omdb_movies_df.loc[index,'Writer']
    Actors = omdb_movies_df.loc[index,'Actors']
    Plot = omdb_movies_df.loc[index,'Plot']
    Language = omdb_movies_df.loc[index,'Language']
    Country = omdb_movies_df.loc[index,'Country']
    Awards = omdb_movies_df.loc[index,'Awards']
    Poster = omdb_movies_df.loc[index,'Poster']
    Ratings = omdb_movies_df.loc[index,'Ratings']
    Metascore = omdb_movies_df.loc[index,'Metascore']
    imdbRating = omdb_movies_df.loc[index,'imdbRating']
    imdbVotes = omdb_movies_df.loc[index,'imdbVotes']
    imdbID = omdb_movies_df.loc[index,'imdbID']
    Type = omdb_movies_df.loc[index,'Type']
    DVD = omdb_movies_df.loc[index,'DVD']
    BoxOffice = omdb_movies_df.loc[index,'BoxOffice']
    Production = omdb_movies_df.loc[index,'Production']
    Website = omdb_movies_df.loc[index,'Website']
    Response = omdb_movies_df.loc[index,'Response']
    Error = omdb_movies_df.loc[index,'Error']
    totalSeasons = omdb_movies_df.loc[index,'totalSeasons']
    print(Title)

    # Add data to database
    session.add(Omdb(
        id = id,
        Title =  Title,
        Year =   Year,
        Rated =   Rated,
        Released =  Released,
        Runtime =  Runtime,
        Genre =   Genre,
        Director =  Director,
        Writer =   Writer,
        Actors =   Actors,
        Plot =   Plot,
        Language =  Language,
        Country =   Country,
        Awards =   Awards,
        Poster =   Poster,
        Ratings = Ratings,
        Metascore =  Metascore,
        imdbRating =  imdbRating,
        imdbVotes = imdbVotes,
        imdbID = imdbID,
        Type = Type,
        DVD =   DVD,
        BoxOffice =  BoxOffice,
        Production = Production,
        Website =   Website,
        Response =  Response,
        Error =  Error,
        totalSeasons = totalSeasons

    ))
print("ok")

Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent
Maleficent

In [33]:
# Commit all changes to database
session.commit()

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) datatype mismatch
[SQL: INSERT INTO movies (movieid, title, mpaa_rating, budget, gross, release_date, genre, runtime, rating, rating_count, summary) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((1, "Look Who's Talking", 'PG-13', 7500000, 296000000, '12/10/1989', 'Romance', 93, 5.9, 73638.0, 'After a single, career-minded woman is left on her own to give birth to the child of a married man, she finds a new romantic chance in a cab driver. Meanwhile, the point-of-view of the newborn boy is narrated through voice-over.'), (2, 'Driving Miss Daisy', 'PG', 7500000, 145793296, '13/12/1989', 'Comedy', 99, 7.4, 91075.0, 'An old Jewish woman and her African-American chauffeur in the American South have a relationship that grows and improves over the 

In [7]:
session.close()